In [4]:
import pandas as pd
import numpy as np
first_data = pd.read_csv('properties_2016.csv')
second_data = pd.read_csv('properties_2017.csv')
initial_data = pd.concat([first_data,second_data], axis=0)
zillow_total_log = pd.read_csv('train_2016_v2.csv')
zillow_total_log_2 = pd.read_csv('train_2017.csv')
initial_data_2 = pd.concat([zillow_total_log,zillow_total_log_2], axis=0)
total_data = pd.merge(initial_data, initial_data_2, on = 'parcelid')

/Users/sahilsobti/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/sahilsobti/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#Filling the missing values 
features = ['bathroomcnt','finishedsquarefeet12','yearbuilt', 
            'structuretaxvaluedollarcnt','landtaxvaluedollarcnt', 'regionidcounty']
q1_data = total_data[features]
q1_data = q1_data.dropna(how='all')
grouped_col = q1_data.groupby("regionidcounty")
q1_data['yearbuilt'] = grouped_col.yearbuilt.transform(lambda x: x.fillna(x.mode()[0]))
q1_data['bathroomcnt'] = grouped_col.bathroomcnt.transform(lambda x: x.fillna(x.mode()[0]))
q1_data['finishedsquarefeet12'] = grouped_col.finishedsquarefeet12.transform(lambda x: x.fillna(x.mode()[0]))
q1_data['structuretaxvaluedollarcnt'] = grouped_col.structuretaxvaluedollarcnt.transform(lambda x: x.fillna(x.mean()))
q1_data['landtaxvaluedollarcnt'] = grouped_col.landtaxvaluedollarcnt.transform(lambda x: x.fillna(x.mean()))
q1_data['yearbuilt'] = 2017 - q1_data['yearbuilt']
q1_data['finishedsquarefeet12'] = np.log(q1_data['finishedsquarefeet12'])
q1_data['structuretaxvaluedollarcnt'] = np.log(q1_data['structuretaxvaluedollarcnt'])
q1_data['landtaxvaluedollarcnt'] = np.log(q1_data['landtaxvaluedollarcnt'])

In [6]:
from sklearn import preprocessing
minn = 1
maxx = 10
min_max_scaler = preprocessing.MinMaxScaler(feature_range =(minn, maxx))

bathroomcnt_x = q1_data.bathroomcnt.values.reshape(-1,1) #returns a numpy array
finishedsquarefeet12_x = q1_data.finishedsquarefeet12.values.reshape(-1,1) #returns a numpy array
structuretaxvaluedollarcnt_x = q1_data.structuretaxvaluedollarcnt.values.reshape(-1,1) #returns a numpy array
landtaxvaluedollarcnt_x = q1_data.landtaxvaluedollarcnt.values.reshape(-1,1) #returns a numpy array

bathroomcnt_x_scaled = min_max_scaler.fit_transform(bathroomcnt_x)
finishedsquarefeet12_x_scaled = min_max_scaler.fit_transform(finishedsquarefeet12_x)
structuretaxvaluedollarcnt_x_scaled = min_max_scaler.fit_transform(structuretaxvaluedollarcnt_x)
landtaxvaluedollarcnt_x_scaled = min_max_scaler.fit_transform(landtaxvaluedollarcnt_x)

q1_data.bathroomcnt = bathroomcnt_x_scaled
q1_data.finishedsquarefeet12 = finishedsquarefeet12_x_scaled
q1_data.structuretaxvaluedollarcnt = structuretaxvaluedollarcnt_x_scaled
q1_data.landtaxvaluedollarcnt = landtaxvaluedollarcnt_x_scaled

In [7]:
def normalizeyears(x):
    if x <= 7 and x >= 0:
        return 10
    elif x > 7 and x <= 17:
        return 7
    elif x > 17 and x <= 80:
        return 5
    elif x > 80 and x <= 90:
        return 8
    elif x > 90 and x <= 100:
        return 9
    elif x >100:
        return 10
def countynormalize(x):
    # fn = 200/(1+ 10*VC + PC)
    #Ventura = 2061
    #VC = 2.3 and PC = 20.1
    #LA = 3101
    #VC = 4.3 and PC = 21.8
    #Orange = 1286
    #VC = 2.0 and PC = 17.7
    if x == 2061:
        r = 4.535
    elif x == 3101:
        r = 3.03
    #Orange
    elif x == 1286:
        r = 5.16
    return r    

In [8]:
yearbuilt_series = q1_data.yearbuilt.apply(normalizeyears)
q1_data.yearbuilt = yearbuilt_series.to_frame(name= 'yearbuilt')

regionidcounty_series = q1_data.regionidcounty.apply(countynormalize)
q1_data.regionidcounty = regionidcounty_series.to_frame(name= 'regionidcounty')

In [9]:
def scoringfunction(row):
    denominator = np.log2(row.structuretaxvaluedollarcnt) * np.log2(row.landtaxvaluedollarcnt)
    result = (row.bathroomcnt * np.log2(row.finishedsquarefeet12) * row.regionidcounty * np.sqrt(row.yearbuilt) ) 
    / np.sqrt(denominator)
    return result

In [10]:
q1_data['score'] = q1_data.apply(scoringfunction, axis = 1)
sorted_by_score = q1_data.sort_values(by = 'score', ascending=False)
sorted_by_score.head()

,bathroomcnt,finishedsquarefeet12,yearbuilt,structuretaxvaluedollarcnt,landtaxvaluedollarcnt,regionidcounty,score
61989,5.725,9.236522,10,9.191004,8.531154,5.16,299.620487
61988,5.050,9.236522,10,9.180338,8.521834,5.16,264.294054
198093,9.100,9.843509,8,6.507387,10.000000,3.03,257.296610
198092,9.100,9.843509,8,6.496727,9.990680,3.03,257.296610
119979,4.825,8.952119,10,8.826954,8.538392,5.16,248.966185


In [12]:
q1_data.iloc[0:100,:]

,bathroomcnt,finishedsquarefeet12,yearbuilt,structuretaxvaluedollarcnt,landtaxvaluedollarcnt,regionidcounty,score
0,3.250,8.293936,5,7.572626,6.984008,4.535,100.586173
1,3.250,8.293936,5,7.583290,6.993327,4.535,100.586173
2,1.900,7.368046,5,6.625208,6.864140,4.535,55.513891
3,1.900,7.368046,5,6.635866,6.873458,4.535,55.513891
4,1.675,7.524812,5,6.900606,6.133413,4.535,49.455786
5,1.675,7.524812,5,6.911271,6.142721,4.535,49.455786
6,2.125,7.239238,5,6.545435,6.022975,4.535,61.539617
7,2.125,7.239238,5,6.556094,6.032287,4.535,61.539617
8,1.900,6.998933,5,6.364292,6.498227,4.535,54.085294
9,1.900,6.998933,5,6.374955,6.507545,4.535,54.085294
